In [53]:
import numpy as np
import gymnasium as gym
import mo_gymnasium as mo_gym
import torch as th
import time
from mo_gymnasium import MORecordEpisodeStatistics
from torch import nn
from morl_baselines.common.networks import mlp


from morl_baselines.single_policy.ser import mo_ppo
# Define the make_env function
def make_env(env_id, seed, idx, run_name, gamma):
    def thunk():
        env = mo_gym.make(env_id)
        env = MORecordEpisodeStatistics(env)
        env.action_space.seed(seed + idx)
        env.observation_space.seed(seed + idx)
        return env
    return thunk

# Step 1: Create the MO Gym environment
env_id = "mo-halfcheetah-v4"
num_envs = 8
seed = 123
gamma = 0.99
envs = gym.vector.SyncVectorEnv(
    [make_env(env_id, seed, idx, f"run_{seed}", gamma) for idx in range(num_envs)]
)

# Step 2: Initialize the agent
obs_shape = envs.single_observation_space.shape
action_shape = envs.single_action_space.shape
#reward_dim = envs.single_reward_space.shape[0]


c:\Users\thoma\anaconda3\envs\MORL_baseline\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.reward to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.reward` for environment variables or `env.get_wrapper_attr('reward')` that will search the reminding wrappers.
  logger.warn(


AttributeError: 'MOHalfCheehtahEnv' object has no attribute 'reward'

In [62]:
# Create a single environment instance to get reward space information
single_env = mo_gym.make(env_id)
single_env.reset()
obs, reward, terminated, truntuated, info = single_env.step(action=(1,1,1,1,1,1))
reward_dim = reward.shape[0]

In [63]:


network = MOPPONet(obs_shape, action_shape, reward_dim)
weights = np.array([1.0] * reward_dim)  # Example weights, can be adjusted

agent = MOPPO(
    id=1,
    networks=network,
    weights=weights,
    envs=envs,
    log=False,
    steps_per_iteration=2048,
    num_minibatches=32,
    update_epochs=10,
    learning_rate=3e-4,
    gamma=0.99,
    anneal_lr=False,
    clip_coef=0.2,
    ent_coef=0.0,
    vf_coef=0.5,
    clip_vloss=True,
    max_grad_norm=0.5,
    norm_adv=True,
    target_kl=None,
    gae=True,
    gae_lambda=0.95,
    device="cpu",  # Adjust based on your hardware
    seed=seed,
)

# Step 3: Train the agent
max_iterations = 1000
start_time = time.time()

for iteration in range(max_iterations):
    agent.train(start_time, current_iteration=iteration, max_iterations=max_iterations)
    if iteration % 10 == 0:
        print(f"Iteration {iteration} completed")

# Step 4: Evaluate the agent
obs = envs.reset(seed=seed)
done = False
total_reward = np.zeros(reward_dim)

while not done:
    action = agent.eval(obs, weights)
    obs, reward, done, _, info = envs.step(action)
    total_reward += reward

print("Total Reward:", total_reward)


ValueError: setting an array element with a sequence.